## 简介
MLlib设计理念是将数据以RDD的形式表示，然后在分布式数据集上调用各种算法。其实，MLlib就是RDD上一系列可供调用的函数的集合

##数据类型

    1. org.apache.spark.mllib（java/scala）
    2. pysqark.mllib(python版本)

### Vector向量

本地向量local vector，索引从0开始，并且是整数，值是double类型，储存在单个机器中，稠密向量表示向量的每一位都储存，后者只储存非零位以节约空间
，通过mllib.linalg.Vectors类创建

In [ ]:
from pyspark.mllib.linalg import Vectors
den=Vectors.dense([1.0,2.0,3.0])
print(den)

'''
SparkVector创建方式有两种：
1.Vector.sparse(向量长度，索引数组，与索引数组所对应的数值数组)
2.Vector.sparse(向量长度，(索引，数值)，(索引，数值)。。。。)

'''

spa=Vectors.sparse(4,[0,2],[1.0,2.0])

### LabeledPoint
在监督学习算法中使用supervised learing，LabeledPoint表示带标签的数据点，包括一个特征向量与一个标签（有一个浮点数表示），位于mllib.regression包里

In [ ]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linelg import Vectors
pos=LabeledPoint(1.0,Vectors.dense([1.0,2.0,3.0]))
neg=LabeledPoint(0.0,Vectors.dense([1.0,2.0,3.0]))

## Matrix

1. 稠密矩阵：实体值以列为主要次序的形式，存放在单个Double型数组中
2. 

### Rating
用于产品推荐，表示用户对一个产品的评分，位于mllib.recommendation包中

## 各类Model类（模型）

每个Model都是训练算法的结果，每个模型都有一个predict（）方法

## 统计
不论是在即时的探索中，还是在机器学习的数据理解中，基本的统计都是数据分析的重要部分。MLlib 通过mllib.stat.Statistics 类中的方法提供了几种广泛使用的统计函数，这些函数可以直接在RDD 上使用。



## 算法


    1.特征提取
    2.降维
    3.分类与回归
    4.聚类
    5.协同过滤和推荐

In [ ]:
#加载模块
from pyspark.mllib.util import MLUtils
from pyspark.mllib.Classification import SVMWithSGD

#读取数据
dataFile='filepath'
data=MLUtils.loadlibSVMFile(sc,dataFile)


#划分数据集
splits=data.randomSplit([0.8,0.2],seed=9L)
training=splits[0].cache()
test=splits[1]

#打印分割后的数据集
print('TrainingCount:%s' % training.count())
print('TestingCount:%s' % test.count())

#构建模型
model=SVMWithSGD.train(training,100)
scoreAndLables=test.map(lambda point:(model.predict(point.features),point.label))
#输出结果，包含预测的数字结果和0/1结果：
for score,label in scoreAndLables:
    print (score ,label)

In [ ]:
#每次使用Spark都需要先构建SparkSession，因此我们导入pyspark.sql库并初始化一个SparkSession
from pyspark.sql import SparkSession

spark=SparkSession.builder.getOrCreate()
#加载数据
data =spark.read.csv('./boston_housing.csv',header=True,inferSchema=True)

In [ ]:
#创建特征数组，只需要引入VectorAssembler类并传入特征向量的列名称即可
feature_columns=data.columns[:-1]   
from pyspark.ml.feature import VectorAssembler
assemler=VectorAssembler(inputCols=feature_columns,outputCol='features')
data_2=assemler.transform(data)

#训练集和测试集的划分，这里使用RandomSplit函数
train,test=data_2.randomSplit([0.7,0.3])

训练与评估模型，与平时我们训练和评估模型一样，只不过在spark中我们使用的是spark为我们提供的算法函数，
在spark中我们需要从pyspark.ml中导入算法函数，使用model.transform()函数进行预测，这个和之前用的model.predict()还是有区别的

In [ ]:
from pyspark.ml.regression import LinearRegression
algo=LinerRegression(featureCol='features',labelCol='medv')
model=algo.fit(train)
evalution_summary=model.evaluate(test)
predictions=model.transfrom(test)

## 完整代码

In [ ]:
# 首先导入findspark库并通过传递Apache Spark文件夹的路径进行初始化。
import findspark
findspark.init('/opt/spark')

#create a sparksession
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()
#load data
data=spark.read.csv('./boston_housing.csv',header=True,inferSchema=True)

#creat features vectors
from pyspark.ml.feature import VectorAssembler
feature_columns = data.columns[:-1]
assembler = VectorAssembler(inputCols=feature_columns,outputCol="features")
data_2 = assembler.transform(data)

#train/test split
train, test = data_2.randomSplit([0.7, 0.3])

# define the model
from pyspark.ml.regression import LinearRegression
algo = LinearRegression(featuresCol="features", labelCol="medv")

# train the model
model = algo.fit(train)
# evaluation
evaluation_summary = model.evaluate(test)
evaluation_summary.meanAbsoluteError
evaluation_summary.rootMeanSquaredError
evaluation_summary.r2
# predicting values
predictions = model.transform(test)
predictions.select(predictions.columns[13:]).show() # here I am filtering out some columns just for the figure to fit